In [42]:
import numpy as np
np.random.seed(21)

In [43]:
## 
def random_plane(labels, space):
    l = len(labels)
    
    random_versor = np.random.uniform(size=l)
    random_versor /= np.linalg.norm(random_versor)

    mean_vect = np.zeros(5)

    for i in range(l):
        mean_vect[i] = (np.max(space[i]) - np.min(space[i]))/2

    print("Random", random_versor)
    print("Mean", mean_vect)
    return random_versor, mean_vect

In [5]:
labels = [1,2,3,4,5]
space = []
space.append([1,2,3])
space.append([1,2,3,4,5,6])
for i in range(3):
    space.append(np.linspace(0,1,10))

In [6]:
r = random_plane(labels, space)

Random [0.06049992 0.358977   0.89519781 0.02684011 0.25568684]
Mean [1.  2.5 0.5 0.5 0.5]


In [7]:
def target_cast (inputs, r_plane):
    targets = []
    for i in range(len(inputs)):
        target =  np.sign(np.dot(inputs[i] - r_plane[1], r_plane[0]))
        if target < 0: target = 0
        targets.append(int(target))
    return targets 

In [8]:
inputs = []
inputs.append([1,2,3]*10)
inputs.append([1,2,3,4,5,6]*5)
for i in range(3):
    inputs.append(np.linspace(0,1,30))

inputs = np.asarray(inputs).T
print(np.shape(inputs))

#target_cast(inputs ,r)

(30, 5)


In [44]:
def target_cast(inputs, r_plane, irrelevant_components=0):
    targets = []
    #print(inputs[0])
    #print(len(r_plane))

    admitted = np.ones(len(r_plane[0]))
    admitted[irrelevant_components] = 0
    for i in range(len(inputs)):
        
        target =  (np.dot(inputs[i] - r_plane[1]*admitted, r_plane[0]*admitted))

        if target < 0: target = 0
        else: target = 1
        targets.append(int(target))
    return np.array(targets, dtype = np.int)

def random_plane(labels, space, irrelevant_components=0,_plot=False):
    print(len(labels))
    l = len(labels)

    random_versor = np.random.uniform(size=l)
    random_versor /= np.linalg.norm(random_versor)

    mean_vect = np.zeros(l)

    for i in range(l):
        mean_vect[i] = np.mean(space[:,i])
        
    print("Random", random_versor)
    print("Mean", mean_vect)

    
    if _plot:
        f = plt.figure(figsize=(18, 10))  # plot the calculated values
        for i in range(l):
            f.add_subplot(2, 3, i+1)
            z = space[:,i]
            plt.hist(z, int(len(z)/100), label='component-%i'%i)
            plt.legend()
        plt.show()

    return [random_versor, mean_vect]

In [45]:
import numpy as np
import matplotlib.pyplot as plt

N = 10**5
label_idx = [0,1,2,3,4,5]

input = np.zeros((N,6))
input[:,0] = np.random.uniform(0,10,N)
input[:,1:] = np.random.normal(1, 2, size=(N,5))

In [46]:
y = target_cast(input, r_plane=random_plane(label_idx, input))

6
Random [0.12799566 0.63937217 0.68556487 0.31038409 0.08828428 0.02624414]
Mean [5.00574328 1.00526396 0.99392551 0.98756058 1.00321675 0.99268155]


In [36]:
len(y[y==0])/len(y)


0.49967

In [37]:
import torch
from sklearn.linear_model import LogisticRegression

In [38]:
in_data = input
y = target_cast(input, r_plane=random_plane(label_idx, input))
lr = LogisticRegression()
lr.fit(in_data, y)
y_pred1 = lr.predict_proba(in_data)            
y_pred1 = torch.tensor(y_pred1, dtype=torch.float)
y = torch.tensor(y, dtype=torch.long)
#y = torch.randint(low=0, high=1, size=(N,) )

torch.nn.CrossEntropyLoss(reduction='mean')(y_pred1, y)

6
Random [0.56867597 0.46563253 0.20038107 0.33545856 0.28420697 0.47574716]
Mean [5.00165462 1.00433513 0.99153517 0.99444646 1.00565136 1.00415663]


/home/emanuele/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


tensor(0.32)

In [39]:
torch.set_printoptions(precision=2, sci_mode=False)
y_pred1.to(dtype=torch.float)

tensor([[    0.00,     1.00],
        [    1.00,     0.00],
        [    0.00,     1.00],
        ...,
        [    0.97,     0.03],
        [    0.00,     1.00],
        [    0.00,     1.00]])

In [40]:
forward = torch.argmax(y_pred1, dim=1 ) 
len(forward[torch.abs(y-forward) == 1])

80

In [47]:
import torch

In [48]:

classification = torch.nn.Linear(10, 2)
optim = torch.optim.SGD(classification.params(), lr = 0.01, momentum=0.9)

loss = torch.nn.CrossEntropyLoss(reduction="sum")




AttributeError: 'Linear' object has no attribute 'params'